# Import thư viện

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [27]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--start-maximized")
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [28]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

# Thông tin phim

In [29]:
# Mở url
# url = 'https://www.rottentomatoes.com/m/operation_fortune_ruse_de_guerre'
# url = 'https://www.rottentomatoes.com/m/bunker_2022'
url = 'https://www.rottentomatoes.com/m/necesito_dinero'
# url = 'https://www.rottentomatoes.com/m/antman'
driver.get(url)

In [30]:
# Đợi cho đến khi url load xong
element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="container roma-layout__body"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

In [31]:
# Tiêu đề phim
title = soup.select('h1[class="scoreboard__title"]')[0].text
title

'Necesito dinero'

In [32]:
# Điểm đánh giá của nhà phê bình
tomatometer = soup.select('score-board')[0].attrs['tomatometerscore']
tomatometer == ''

True

In [33]:
# Điểm đánh giá của khán giả
audience_score = soup.select('score-board')[0].attrs['audiencescore']
audience_score

''

In [34]:
# Số lượt đánh giá của nhà phê bình
num_tomatometer = soup.select('a[slot="critics-count"]')[0].text.strip()
num_tomatometer

'0 Reviews'

In [35]:
# Số lượt đánh giá của khán giả
num_audience_score = soup.select('a[slot="audience-count"]')[0].text.strip()
num_audience_score

'250+ Ratings'

In [36]:
# The Tomatometer score
tomatometer_state = soup.select('score-board')[0].attrs['tomatometerstate']
tomatometer_state

''

In [37]:
# The Audience Score
audience_state = soup.select('score-board')[0].attrs['audiencestate']
audience_state

''

In [52]:
# Thông tin phim
list_name_info = soup.select('b[class="info-item-label"]')
list_info = soup.select('span[data-qa="movie-info-item-value"]')

for num_info in range(len(list_name_info)):
    name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    print(name_info + info)

GenreRomance, Comedy
Original LanguageEnglish
DirectorMiguel Zacarías
WriterEdmundo Báez, Miguel Zacarías
Runtime1h 30m


In [39]:
# Tóm tắt nội dụng phim
synopsis = soup.select('p[data-qa="movie-info-synopsis"]')[0].text.strip()
synopsis

'A poor mechanic falls in love with a beautiful and ambitious girl who passed through the workshop where he works.'

In [51]:
# Diễn viên
list_raw_cast = soup.select('a[data-qa="cast-crew-item-link"] > p')
list_cast = []

for c in range(len(list_raw_cast)):
    list_cast.append(list_raw_cast[c].text)

cast = ', '.join(list_cast)
cast

'Pedro Infante, Sara Montiel, Irma Dorantes, Gustavo Rivero, Armando Sáenz, Elda Peralta, Guillermo Samperio, Gloria Morel, Luis Mussot, Armando Velasco, María España Vidal, Miguel Zacarías, Edmundo Báez, Miguel Zacarías, Jorge Stahl Jr., Manuel Esperón'

# Reviews

### Critic and Top critic reviews

In [ ]:
# Chuyển đến url critic reviews
driver.get(url + '/reviews')

element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

In [ ]:
# Check có review của nhà phê bình không ?
check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

if check_critic_review != '':
    list_critic_review = []
    list_top_critic_review = []
    
    # Số lượng review trong 1 trang
    num_critic_rv = len(soup_critic_review.select('div[class="review-row"]'))

    for num in range(num_critic_rv):
        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
        list_critic_review.append(rv)

        # Check top critic reviews
        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

        if check_top_critic != []:
            list_top_critic_review.append(rv)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
            soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

            num_critic_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_critic_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                list_critic_review.append(rv)

                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
        except Exception:
            # traceback.print_exc()
            break

    critic_review = '<>'.join(list_critic_review)
    if list_top_critic_review != []:
        top_critic_review = '<>'.join(list_top_critic_review)
    else:
        top_critic_review = 'No top critic reviews'
    
    print(critic_review)
    print(top_critic_review)
else:
    critic_review = 'No critic reviews'
    print(critic_review)

Feels like the third or fourth installment of a forgotten franchise, one of those late-series entries where everybody’s happy to see each other again and nobody is going to be accused of over-exerting themselves. This is not a knock. <>A serviceable chunk of entertainment that will probably work best with lowered expectations. <>“Operation Fortune” should be a Hartnett/Grant movie, not a Statham one. This snarky action comedy shines brightest when Grant and Hartnett are on screen.<>Ritchie decides that telling is more important than showing in this overly dry and formulaic take on the spy thriller. Statham is passable as yet another variant of his various other onscreen action personas, but it's sad to see Plaza wasted in her role. <>There’s less action than one might expect and the character development is so thin that even when a character is in danger, it’s hard to care.<>Everybody in Operation Fortune — yes, even Ritchie — seems to be having fun. Sometimes, that’s all you need.<>Op

### Audience and Verified audience reviews

In [ ]:
# Chuyển đến url review của khán giả
driver.get(url + '/reviews?type=user')

element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

In [ ]:
# Check có review của khán giả không ?
check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

if len(check_audience_review) != 0:
    list_audience_review = []
    list_verified_audience_review = []

    # Số lượng review trong 1 trang
    num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
    
    for num in range(num_audience_rv):
        rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
        list_audience_review.append(rv)

        # Check verified audience review
        check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

        if check_verified_audience != []:
            list_verified_audience_review.append(rv)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML', element_audience_review)
            soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

            # Số lượng review trong 1 trang
            num_audience_rv = len(soup_audience_review.select('div[class="audience-review-row"]'))
            
            for num in range(num_audience_rv):
                rv = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('p', {'data-qa': 'review-text'})[0].text
                list_audience_review.append(rv)

                # Check verified audience review
                check_verified_audience = soup_audience_review.select('div[class="audience-review-row"]')[num].find_all('span', {'class': 'audience-reviews__verified'})

                if check_verified_audience != []:
                    list_verified_audience_review.append(rv)
            
        except:
            break
    audience_review = '<>'.join(list_audience_review)
    if list_verified_audience_review != []:
        verified_audience_review = '<>'.join(list_verified_audience_review)
    else:
        verified_audience_review = 'No verified audience reviews'
    print(verified_audience_review)
    
else:
    audience_review = 'No audience reviews'
    verified_audience_review = 'No verified audience reviews'
    print(audience_review)

This was a Guy Ritchie masterpiece.<>Very entertaining. Well worth the price of the ticket. Lots of action. Great scene visuals. Grantham and Grant were great. wonderful to watch. would see it again.<>If you like action movies with ridiculous fight and shoot-out scenes, you will like it. If you like real movies, you will wonder why you bothered. I just don't get these kind of movies. It's mind-blowing to me how popular they are.<>It was a good enough movie, the action bits were cool, and two of my favorite actors just topped it for me. It would’ve been better if the plot was a bit more detailed, i guessed out who the antagonist mid way through and just caved in the plot from there. it was bit funny though, i’ll give it that.<>This was a Guy Ritchie masterpiece.<>Very entertaining. Well worth the price of the ticket. Lots of action. Great scene visuals. Grantham and Grant were great. wonderful to watch. would see it again.<>If you like action movies with ridiculous fight and shoot-out s